In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pyvis.network import Network
import networkx as nx

In [3]:
import sys
 
# setting path
sys.path.append('../')
from solver import _data_collector

# Read and visualize Graphs

In [4]:
def read_file(file_location: str) -> str:
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
    
    edges, edge_count, node_count = _data_collector(input_data)
    
    return edges, edge_count, node_count

def to_network(edges: list, node_count: int, colors: list = None) -> Network:
    # Plot
    g_plot = Network(notebook=True, cdn_resources="remote")
    g_plot.repulsion()
    
    # Graph
    g = nx.graph.Graph()
    for node in range(node_count):
        color = 0
        if colors:
            color = colors[node]
        g.add_node(node, title=f"Node {node}", group=color)
    
    for edge in edges:
        g.add_edge(edge[0], edge[1])
        
    g_plot.from_nx(g)
    
    return g_plot, g

In [5]:
edges, edge_count, node_count = read_file("../data/gc_20_3")

In [6]:
n_plot, network = to_network(edges, node_count)

In [7]:
n_plot

<class 'pyvis.network.Network'> |N|=20 |E|=63

In [8]:
n_plot.show("Network.html")

Network.html


In [11]:
nx.find_cliques(network).__next__()

[0, 4]

# Solving coloring problem

In [9]:
from ortools.sat.python import cp_model

In [10]:
model = cp_model.CpModel()

In [11]:
# Create variables
nodes = [
    model.NewIntVar(0, node_count-1, 'x%i' % i) for i in range(node_count)
]

# Variable of the objective function - Its the maximum number of colors
final_value = model.NewIntVar(0, node_count-1, 'max color')

In [12]:
edges

[(0, 16),
 (1, 2),
 (1, 6),
 (1, 7),
 (1, 8),
 (2, 11),
 (2, 16),
 (2, 17),
 (3, 14),
 (3, 16),
 (3, 17),
 (4, 7),
 (4, 13),
 (4, 17),
 (5, 6),
 (5, 11),
 (6, 18),
 (9, 12),
 (10, 13),
 (11, 17),
 (13, 15),
 (15, 17),
 (16, 19)]

In [13]:
# Add constraints
for i, j in edges:
    model.Add(nodes[i]!=nodes[j])

In [14]:
model.AddMaxEquality(final_value, nodes)
model.Minimize(final_value)

In [15]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [16]:
solver.BestObjectiveBound()

2.0

In [17]:
solver.StatusName()

'OPTIMAL'

In [18]:
solver.Value(nodes[2])

1

In [19]:
colors = [solver.Value(node) for node in nodes]

In [20]:
colors

[1, 2, 1, 1, 1, 2, 1, 0, 1, 2, 1, 0, 0, 0, 2, 1, 0, 2, 0, 1]

## Print colored graph

In [21]:
colors

[1, 2, 1, 1, 1, 2, 1, 0, 1, 2, 1, 0, 0, 0, 2, 1, 0, 2, 0, 1]

In [22]:
color_n_plot, color_network = to_network(edges, node_count, colors)

In [23]:
color_n_plot.show("ColorNetwork.html")

ColorNetwork.html


In [37]:
color_network.edges

EdgeView([(0, 16), (1, 2), (1, 6), (1, 7), (1, 8), (2, 11), (2, 16), (2, 17), (3, 14), (3, 16), (3, 17), (4, 7), (4, 13), (4, 17), (5, 6), (5, 11), (6, 18), (9, 12), (10, 13), (11, 17), (13, 15), (15, 17), (16, 19)])

In [40]:
l = [i for i in color_network.nodes]

In [44]:
import random

In [45]:
random.shuffle(l)

In [46]:
l

[1, 11, 5, 2, 8, 16, 14, 6, 12, 17, 7, 15, 10, 0, 9, 18, 4, 19, 3, 13]

# Find cliques

In [30]:
nx.find_cliques(color_network)

<generator object find_cliques at 0x136383900>

In [34]:
nodes[0].Index()

0

In [32]:
for i in nx.find_cliques(color_network):
    print(i)

[0, 16]
[1, 8]
[1, 2]
[1, 6]
[1, 7]
[5, 11]
[5, 6]
[6, 18]
[7, 4]
[9, 12]
[10, 13]
[13, 4]
[13, 15]
[14, 3]
[16, 19]
[16, 2]
[16, 3]
[17, 2, 11]
[17, 3]
[17, 4]
[17, 15]


In [29]:
color_network.clique

AttributeError: 'Graph' object has no attribute 'clique'

# Add Symmetry breaking and Global constraints

In [115]:
model = cp_model.CpModel()

In [116]:
# Create variables
## Nodes Breaking Symmetry (Node 0 -> 1 color, Node 2 -> 2 colors, Node 3 -> 3 colors...)
nodes = [
    model.NewIntVar(0, i, f"Edge {i}") for i in range(node_count)
]
## Variable of the of the objective function
## Objetive function is the max number of the nodes
obj_func = model.NewIntVar(0, node_count-1, "Max color")
model.AddMaxEquality(target=obj_func, exprs=nodes)

In [117]:
# Add constraints
## Adjacente nodes must be different one by one
for i,j in edges:
    model.Add(nodes[i] != nodes[j])

In [100]:
node_cardinality = list()
for node in range(node_count):
    set_adj_nodes = set([i for edge in edges for i in edge if node in edge])
    cardinality = len(set_adj_nodes) - 1
    node_cardinality.append((node, cardinality))

In [101]:
node_cardinality = sorted(node_cardinality, key=lambda x: x[1], reverse=True)

In [118]:
model.AddDecisionStrategy(nodes, cp_model.CHOOSE_FIRST, cp_model.SELECT_MIN_VALUE)

In [119]:
## Global constraints - Adjacent nodes must be all-different
for node in range(node_count):
    set_adj_nodes = set([i for edge in edges for i in edge if node in edge])
    _nodes = list(filter(lambda x: x.Index() in set_adj_nodes, nodes))
    model.AddAllDifferent(_nodes)

In [120]:
set_adj_nodes

{16, 19}

In [121]:
node

19

In [122]:
_nodes

[Edge 16(0..16), Edge 19(0..19)]

In [123]:
model.Minimize(obj_func)

In [124]:
# Solve
solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 10
solver.Solve(model)

4

In [125]:
solver.BestObjectiveBound()

5.0

In [126]:
colors = [solver.Value(node) for node in nodes]

In [127]:
colors

[0, 0, 2, 3, 4, 3, 4, 3, 1, 1, 1, 5, 0, 2, 0, 0, 4, 1, 1, 1]

In [128]:
color_network = to_network(edges, node_count, colors)

In [129]:
color_network.show("ColorNetwork.html")

ColorNetwork.html
